<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/FineTune_Llama2/Forecast_DaysToSettle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install --no-cache-dir -qU \
    jsonlines datasets transformers bitsandbytes accelerate peft trl

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
import transformers

base_model = "meta-llama/Llama-2-7b-chat-hf"

pretrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [4]:
prompt = """
Given the dataset delimited by the triple backticks, forecast number of days \
client 2621-XCLEH will take for the payment of an \
invoice dated 2013-07-28 with an amount $92.17 to be settled. 
Return the forecast in JSON format, containing four keys: \
'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'.
```
customerID InvoiceDate InvoiceAmount DaysToSettle
2621-XCLEH  2012-01-13         80.99      61
2621-XCLEH  2012-02-21         79.51      46
2621-XCLEH  2012-02-22         69.80      43
2621-XCLEH  2012-03-02         67.51      57
2621-XCLEH  2012-03-23         89.05      52
2621-XCLEH  2012-04-16         74.06      44
2621-XCLEH  2012-06-27         69.42      60
2621-XCLEH  2012-11-18         86.39      75
2621-XCLEH  2013-03-01         58.96      55
2621-XCLEH  2013-03-31         70.93      55
2621-XCLEH  2013-04-27         65.76      37
2621-XCLEH  2013-06-18         37.49      29
2621-XCLEH  2013-06-24         90.62      35
2621-XCLEH  2013-07-16         78.08      48
```
"""

pipe = transformers.pipeline(
    task="text-generation", 
    model=pretrained_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<s>[INST] 
Given the dataset delimited by the triple backticks, forecast number of days client 2621-XCLEH will take for the payment of an invoice dated 2013-07-28 with an amount $92.17 to be settled. 
Return the forecast in JSON format, containing four keys: 'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'.
```
customerID InvoiceDate InvoiceAmount DaysToSettle
2621-XCLEH  2012-01-13         80.99      61
2621-XCLEH  2012-02-21         79.51      46
2621-XCLEH  2012-02-22         69.80      43
2621-XCLEH  2012-03-02         67.51      57
2621-XCLEH  2012-03-23         89.05      52
2621-XCLEH  2012-04-16         74.06      44
2621-XCLEH  2012-06-27         69.42      60
2621-XCLEH  2012-11-18         86.39      75
2621-XCLEH  2013-03-01         58.96      55
2621-XCLEH  2013-03-31         70.93      55
2621-XCLEH  2013-04-27         65.76      37
2621-XCLEH  2013-06-18         37.49      29
2621-XCLEH  2013-06-24         90.62      35
2621-XCLEH  2013-07-16         78.08 

In [3]:
import os
import pandas as pd

input_dir = "../input/invoice-payment"
input_csv = "WA_Fn-UseC_-Accounts-Receivable.csv"
df_invoice = pd.read_csv(os.path.join(input_dir, input_csv))
df_invoice['InvoiceDate'] = pd.to_datetime(df_invoice['InvoiceDate'],
    format='%m/%d/%Y').dt.date
df_invoice.drop(columns=['countryCode', 'PaperlessDate', 'invoiceNumber', 'DueDate', 
    'Disputed', 'SettledDate', 'PaperlessBill', 'DaysLate'], inplace=True)
df_invoice.sort_values(by=['customerID', 'InvoiceDate'], inplace=True)
df_invoice.reset_index(inplace=True, drop=True)

df_invoice[df_invoice['customerID']=='2621-XCLEH']

,customerID,InvoiceDate,InvoiceAmount,DaysToSettle
401,2621-XCLEH,2012-01-13,80.99,61
402,2621-XCLEH,2012-02-21,79.51,46
403,2621-XCLEH,2012-02-22,69.80,43
404,2621-XCLEH,2012-03-02,67.51,57
405,2621-XCLEH,2012-03-23,89.05,52
406,2621-XCLEH,2012-04-16,74.06,44
407,2621-XCLEH,2012-06-27,69.42,60
408,2621-XCLEH,2012-11-18,86.39,75
409,2621-XCLEH,2013-03-01,58.96,55
410,2621-XCLEH,2013-03-31,70.93,55


In [4]:
import pandas as pd
import jsonlines

def create_inst_resp(x):
    d = {}
    d['instruction'] = f"""\
Given the dataset delimited by the triple backticks, forecast number of days \
client {x['customerID'][-2:-1].values[0]} will take for the payment of an \
invoice dated {x['InvoiceDate'][-2:-1].values[0]} with an \
amount ${x['InvoiceAmount'][-2:-1].values[0]} to be settled. 
Return the forecast in JSON format, containing four keys: \
'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'.
```
{x[:-2].to_string(index=False)}
```\
"""
    d['response'] = f"""\
{{
    "customerID": {x['customerID'][-2:-1].values[0]},
    "InvoiceDate": {x['InvoiceDate'][-2:-1].values[0]},
    "InvoiceAmount": {x['InvoiceAmount'][-2:-1].values[0]},
    "DaysToSettle": {x['DaysToSettle'][-2:-1].values[0]}
}}\
"""

    return pd.Series(d, index=['instruction', 'response'])

df_prompt = df_invoice.copy()\
    .sort_values(by=['customerID', 'InvoiceDate'])\
    .groupby('customerID')\
    .apply(create_inst_resp)\
    .reset_index()\
    .drop(columns=['customerID'])

prompt_template = """<s>[INST]
{instruction}
[/INST]
{response}
</s>"""

prompts = []
for i in range(len(df_prompt)):
    text = prompt_template.format(
        instruction=df_prompt.loc[i, 'instruction'],
        response=df_prompt.loc[i, 'response']
    )
    prompts.append({
        "text": text,
    })

with jsonlines.open('prompt-invoice-payment.jsonl', 'w') as writer:
    writer.write_all(prompts)

In [5]:
import datasets

finetune_dataset = datasets.load_dataset('json',
    data_files='prompt-invoice-payment.jsonl', split='train')
print(finetune_dataset)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 100
})


In [6]:
print(finetune_dataset[17])

{'text': '<s>[INST]\nGiven the dataset delimited by the triple backticks, forecast number of days client 2621-XCLEH will take for the payment of an invoice dated 2013-07-16 with an amount $78.08 to be settled. \nReturn the forecast in JSON format, containing four keys: \'customerID\', \'InvoiceDate\', \'InvoiceAmount\' and \'DaysToSettle\'.\n```\ncustomerID InvoiceDate  InvoiceAmount  DaysToSettle\n2621-XCLEH  2012-01-13          80.99            61\n2621-XCLEH  2012-02-21          79.51            46\n2621-XCLEH  2012-02-22          69.80            43\n2621-XCLEH  2012-03-02          67.51            57\n2621-XCLEH  2012-03-23          89.05            52\n2621-XCLEH  2012-04-16          74.06            44\n2621-XCLEH  2012-06-27          69.42            60\n2621-XCLEH  2012-11-18          86.39            75\n2621-XCLEH  2013-03-01          58.96            55\n2621-XCLEH  2013-03-31          70.93            55\n2621-XCLEH  2013-04-27          65.76            37\n2621-XCLEH  201

In [7]:
import torch
import transformers

base_model = "meta-llama/Llama-2-7b-chat-hf"

quantization_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quantization_config,
    device_map='auto'
)
model.config.use_cache = False

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
import transformers
import peft
import trl

training_args = transformers.TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_steps=15,
    warmup_steps=2
)

peft_config = peft.LoraConfig(
    lora_alpha=64, lora_dropout=0.1, r=16,
    bias='none', task_type='CAUSAL_LM'
)

trainer = trl.SFTTrainer(
    model=model,
    train_dataset=finetune_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

trainer.train()

peft_model = "peft_model"
trainer.model.save_pretrained(peft_model)

del model
del trainer
torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:173: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.103300
2,1.024300
3,1.035600
4,0.988400
5,0.919700
6,0.856000
7,0.826200
8,0.782900
9,0.783400
10,0.748200


In [9]:
import torch
import transformers
import peft

pretrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
finetuned_model = peft.PeftModel.from_pretrained(
    pretrained_model, peft_model
)
finetuned_model = finetuned_model.merge_and_unload()
finetuned_dir = "finetuned_Llama2_forecast_DaysToSettle"
finetuned_model.save_pretrained(finetuned_dir)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(finetuned_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('finetuned_Llama2_forecast_DaysToSettle/tokenizer_config.json',
 'finetuned_Llama2_forecast_DaysToSettle/special_tokens_map.json',
 'finetuned_Llama2_forecast_DaysToSettle/tokenizer.model',
 'finetuned_Llama2_forecast_DaysToSettle/added_tokens.json',
 'finetuned_Llama2_forecast_DaysToSettle/tokenizer.json')

In [11]:
import transformers

prompt = """
Given the dataset delimited by the triple backticks, forecast number of days \
client 2621-XCLEH will take for the payment of an \
invoice dated 2013-07-28 with an amount $92.17 to be settled. 
Return the forecast in JSON format, containing four keys: \
'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'.
```
customerID InvoiceDate InvoiceAmount DaysToSettle
2621-XCLEH  2012-01-13         80.99      61
2621-XCLEH  2012-02-21         79.51      46
2621-XCLEH  2012-02-22         69.80      43
2621-XCLEH  2012-03-02         67.51      57
2621-XCLEH  2012-03-23         89.05      52
2621-XCLEH  2012-04-16         74.06      44
2621-XCLEH  2012-06-27         69.42      60
2621-XCLEH  2012-11-18         86.39      75
2621-XCLEH  2013-03-01         58.96      55
2621-XCLEH  2013-03-31         70.93      55
2621-XCLEH  2013-04-27         65.76      37
2621-XCLEH  2013-06-18         37.49      29
2621-XCLEH  2013-06-24         90.62      35
2621-XCLEH  2013-07-16         78.08      48
```
"""

pipe = transformers.pipeline(
    task="text-generation", 
    model=finetuned_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 
Given the dataset delimited by the triple backticks, forecast number of days client 2621-XCLEH will take for the payment of an invoice dated 2013-07-28 with an amount $92.17 to be settled. 
Return the forecast in JSON format, containing four keys: 'customerID', 'InvoiceDate', 'InvoiceAmount' and 'DaysToSettle'.
```
customerID InvoiceDate InvoiceAmount DaysToSettle
2621-XCLEH  2012-01-13         80.99      61
2621-XCLEH  2012-02-21         79.51      46
2621-XCLEH  2012-02-22         69.80      43
2621-XCLEH  2012-03-02         67.51      57
2621-XCLEH  2012-03-23         89.05      52
2621-XCLEH  2012-04-16         74.06      44
2621-XCLEH  2012-06-27         69.42      60
2621-XCLEH  2012-11-18         86.39      75
2621-XCLEH  2013-03-01         58.96      55
2621-XCLEH  2013-03-31         70.93      55
2621-XCLEH  2013-04-27         65.76      37
2621-XCLEH  2013-06-18         37.49      29
2621-XCLEH  2013-06-24         90.62      35
2621-XCLEH  2013-07-16         78.08 